In [1]:
using COBREXA

Let's download and open a big model

In [2]:
import Downloads
Downloads.download("http://bigg.ucsd.edu/static/models/iJO1366.json", "ecoli.json")
m = load_model(StandardModel, "ecoli.json");

...we've opened it as StandardModel right away to allow easy manual
modifications.

let's have a look at the genes

In [3]:
m.genes["b0722"]

Gene.id: b0722
Gene.name: sdhD
Gene.notes: 
	original_bigg_ids: ["b0722"]
Gene.annotations: 
	sbo: ["SBO:0000243"]
	uniprot: ["P0AC44"]
	ecogene: ["EG10934"]
	ncbigene: ["945322"]
	ncbigi: ["16128697"]
	refseq_locus_tag: ["b0722"]
	refseq_name: ["sdhD"]
	asap: ["ABE-0002464"]
	refseq_synonym: ["JW0712", "ECK0711"]


see the objective function

In [4]:
objective(m) # gives a sparse vector that maximizes a single reaction
reactions(m)[19] # the number may differ

"BIOMASS_Ec_iJO1366_core_53p95M"

see the exchanges.
Unfortunately the standard depends on prefixing the reaction ID with EX_, so
at least we can filter the reactions to them.

In [5]:
exchanges = filter(startswith("EX_"), reactions(m))

exchanges .=> reaction_name.(Ref(m), exchanges)

324-element Vector{Pair{String, String}}:
        "EX_cm_e" => "Chloramphenicol exchange"
       "EX_cmp_e" => "CMP exchange"
       "EX_co2_e" => "CO2 exchange"
   "EX_cobalt2_e" => "Co2+ exchange"
    "EX_colipa_e" => "Core oligosaccharide lipid A exchange"
    "EX_glc__D_e" => "D-Glucose exchange"
      "EX_glcn_e" => "D-Gluconate exchange"
      "EX_glcr_e" => "D-Glucarate exchange"
   "EX_colipap_e" => "Core oligosaccharide lipid A diphosphate exchange"
     "EX_glcur_e" => "D-Glucuronate exchange"
                  ⋮
  "EX_galct__D_e" => "D-Galactarate exchange"
 "EX_galctn__D_e" => "D-Galactonate exchange"
 "EX_galctn__L_e" => "L-Galactonate exchange"
      "EX_galt_e" => "Galactitol exchange"
     "EX_galur_e" => "D-Galacturonate exchange"
       "EX_gam_e" => "D-Glucosamine exchange"
     "EX_gam6p_e" => "D-Glucosamine 6-phosphate exchange"
     "EX_gbbtn_e" => "Gamma-butyrobetaine exchange"
       "EX_gdp_e" => "GDP exchange"

Let's run the FBA. First we need everyone's favorite linear solver
Ref: https://lcsb-biocore.github.io/COBREXA.jl/stable/examples/05a_fba/

In [6]:
using GLPK

For simplicity, let's ask for a dictionary right away.

In [7]:
sol = flux_balance_analysis_dict(m, GLPK.Optimizer)

Dict{String, Float64} with 2583 entries:
  "Zn2tex"        => 0.000334989
  "GUI1"          => 0.0
  "DXYLK"         => 0.0
  "CBL1tonex"     => 0.0
  "FE3DCITtonex"  => 0.0
  "FACOAL180t2pp" => 0.0
  "METSOXR1"      => 0.0
  "LIPOtex"       => 0.0
  "NTD11"         => 0.0
  "GLUNpp"        => 0.0
  "ORNDC"         => 0.0
  "ALAGLUE"       => 0.0
  "UAGCVT"        => 0.0272981
  "I2FE2ST"       => 2.75064e-5
  "BMOCOS"        => 0.000119849
  "GLCURt2rpp"    => 0.0
  "EX_g3pg_e"     => 0.0
  "EX_acolipa_e"  => 0.0
  "PGSA141"       => 0.0
  ⋮               => ⋮

we can have a look at our objective reaction

In [8]:
sol["BIOMASS_Ec_iJO1366_core_53p95M"]  # (it is possible to tab through the dictionary keys)

0.9823718127269752

we can have a look at the main things that happen

In [9]:
flux_summary(sol)

Biomass
  BIOMASS_Ec_iJO1366_core_53p95M: 0.9824
  BIOMASS_Ec_iJO1366_WT_53p95M:   0.0
Import
  EX_o2_e:                        -17.5789
  EX_nh4_e:                       -10.6104
  EX_glc__D_e:                    -10.0
  EX_pi_e:                        -0.9476
  EX_so4_e:                       -0.2478
  EX_k_e:                         -0.1918
  EX_fe2_e:                       -0.0158
  EX_mg2_e:                       -0.0085
  EX_ca2_e:                       -0.0051
  EX_cl_e:                        -0.0051
  EX_cu2_e:                       -0.0007
  EX_mn2_e:                       -0.0007
  EX_zn2_e:                       -0.0003
  EX_ni2_e:                       -0.0003
  EX_mobd_e:                      -0.0001
  EX_cobalt2_e:                   -0.0
Export
  EX_meoh_e:                      0.0
  EX_h_e:                         9.0263
  EX_co2_e:                       19.6752
  EX_h2o_e:                       45.6194


(this guesses the exchange/biomass status based on reaction IDs)

or write the result to a file for future use

In [10]:
using DataFrames, CSV
df = DataFrame(reaction = collect(keys(sol)), flux = collect(values(sol)))
CSV.write("mysolution.csv", df)

"mysolution.csv"

Let's choke the model a bit, reducing the availbale oxygen and sugar

In [11]:
m.reactions["EX_o2_e"].lb = -20
m.reactions["EX_glc__D_e"].lb = -2
sol = flux_balance_analysis_dict(m, GLPK.Optimizer)
sol["BIOMASS_Ec_iJO1366_core_53p95M"] # less growth

0.18602227486090275

allow eating acetate instead

In [12]:
m.reactions["EX_ac_e"].lb = -100;
sol = flux_balance_analysis_dict(m, GLPK.Optimizer);
sol["BIOMASS_Ec_iJO1366_core_53p95M"] # a lot of growth again

0.6895707851538995

at this point, it is clear that the original model data has been overwritten
so it might be much better to do this stuff reproducibly. COBREXA has
modifications for that purpose.

In [13]:
m = load_model(StandardModel, "ecoli.json");
sol = flux_balance_analysis_dict(
    m,
    GLPK.Optimizer,
    modifications = [
        change_constraint("EX_o2_e", lb = -20.0),
        change_constraint("EX_glc__D_e", lb = -2.0),
    ],
);
sol["BIOMASS_Ec_iJO1366_core_53p95M"]
#...this scales much better if you need to do more stuff.

0.18602227486090275

Escher plotting is available via https://github.com/stelmo/Escher.jl

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*